In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers

def get_mnist_model():            
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()    
images = images.reshape((60000, 28 * 28)).astype("float32") / 255 
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255 
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

# Chuẩn bị mô hình và các thành phần cần thiết
model = get_mnist_model()  # Hàm trả về mô hình MNIST
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = keras.metrics.Mean()

# Định nghĩa hàm huấn luyện cho một batch
@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    logs = {}
    for metric in metrics:                           
        metric.update_state(targets, predictions)   
        logs[metric.name] = metric.result()
    
    loss_tracking_metric.update_state(loss)       
    logs["loss"] = loss_tracking_metric.result()  
    return logs  

def reset_metrics():
    for metric in metrics:
        metric.reset_state()
        loss_tracking_metric.reset_state()

training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
training_dataset = training_dataset.batch(32)
epochs = 3 
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"Results at the end of epoch {epoch}")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")



Results at the end of epoch 0
...sparse_categorical_accuracy: 0.9127
...loss: 0.2920
Results at the end of epoch 1
...sparse_categorical_accuracy: 0.9541
...loss: 0.1627
Results at the end of epoch 2
...sparse_categorical_accuracy: 0.9615
...loss: 0.1323


In [15]:
@tf.function
def test_step(inputs, targets):
    predictions = model(inputs, training=False)   
    loss = loss_fn(targets, predictions)
 
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()
 
    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs
        
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics() 
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch) 
print("Evaluation results:") 
for key, value in logs.items(): 
    print(f"...{key}: {value:.4f}")

Evaluation results:
...val_sparse_categorical_accuracy: 0.9697
...val_loss: 0.1170


In [16]:
# the loss_tracker will be used to track the average of per-batch losses during training
loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_tracker = keras.metrics.Mean(name="loss")

# our own Model so can override train_step
class CustomModel(keras.Model):

    # override the train_step method, this is basically the same as was developed
    # in previous example
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            # but using self(inputs, training=True) instead of model() since our
            # model is this class instance itself
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)

        # NOTE: typo in textbook, these need to be self.trainable_weights(),
        # and self.optimizer.apply_gradients()
        # using model.trainable_weights() is referring to an external variable
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))

        # update the loss tracker metric that tracks the average of the loss
        loss_tracker.update_state(loss)
        # return the average loss so far by querying the loss tracker metric
        return {"loss": loss_tracker.result()}

    # any metric you would like to reset across epochs should be listed here
    @property
    def metrics(self):
        return [loss_tracker]

In [17]:
# todo getting "numpy() is only available when eager execution is enabled.", though using TensorFlow 2.x and it appears enabled
# need to debug this example further.
#import tensorflow as tf
#tf.compat.v1.enable_eager_execution()

inputs = keras.Input(shape=(28 * 28,))
features = layers.Dense(512, activation="relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop())
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.2970
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1575
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1308
